In [42]:
%reload_ext autoreload
%autoreload 2

In [1]:
HOPSWORK_PROJECT_NAME = 'forecast_model'

In [12]:
import os
from dotenv import load_dotenv
from src.paths import PARENT_DIR, RAW_DATA_DIR

load_dotenv(PARENT_DIR / '.env')

HOPSWORK_API_KEY  = os.environ['HOPSWORK_API_KEY']

In [3]:
from datetime import datetime
import pandas as pd
from src.data import load_raw_data

# from_year = 2022
# to_year = datetime.now().year
# print(f'Downloading raw data from {from_year} to {to_year}')

# rides = pd.DataFrame()
# for year in range(from_year, to_year+1):

#     rides_one_year = load_raw_data(year)

#     #append rows
#     rides = pd.concat([rides, rides_one_year])

In [4]:
rides = load_raw_data(2024, [4,5,6,7,8])

local file:  C:\Users\ingfz\ML Projects\taxi_demand_predictor\data\raw\rides_2024-04.parquet
File 2024-04 was already in local storage
local file:  C:\Users\ingfz\ML Projects\taxi_demand_predictor\data\raw\rides_2024-05.parquet
File 2024-05 was already in local storage
local file:  C:\Users\ingfz\ML Projects\taxi_demand_predictor\data\raw\rides_2024-06.parquet
2024-06 file is not available
local file:  C:\Users\ingfz\ML Projects\taxi_demand_predictor\data\raw\rides_2024-07.parquet
2024-07 file is not available
local file:  C:\Users\ingfz\ML Projects\taxi_demand_predictor\data\raw\rides_2024-08.parquet
2024-08 file is not available


In [6]:
from src.data import transform_raw_data_into_ts_data

ts_data = transform_raw_data_into_ts_data(rides)

100%|██████████| 261/261 [00:01<00:00, 207.97it/s]


In [25]:
ts_data

,pickup_hour,rides,pickup_location_id
0,2024-04-01 00:00:00,2,10
1,2024-04-01 01:00:00,1,10
2,2024-04-01 02:00:00,0,10
3,2024-04-01 03:00:00,0,10
4,2024-04-01 04:00:00,0,10
...,...,...,...
382099,2024-05-31 19:00:00,0,5
382100,2024-05-31 20:00:00,0,5
382101,2024-05-31 21:00:00,0,5
382102,2024-05-31 22:00:00,0,5


In [8]:
import hopsworks

In [14]:
project = hopsworks.login(
    project=HOPSWORK_PROJECT_NAME,
    api_key_value=HOPSWORK_API_KEY
)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/424087


In [16]:
feature_store = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


In [18]:
FEATURE_GROUP_NAME = 'time_series_hourly_feature_group'
FEATURE_GROUP_VERSION = 1

In [19]:
feature_group = feature_store.get_or_create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    description='Time-series data at hourly frequency',
    primary_key=['pickup_location_id','pickup_hour'],
    event_time='pickup_hour',
)

In [21]:
feature_group.insert(ts_data, write_options={'wait_for_job':False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/424087/fs/419910/fg/1071436


Uploading Dataframe: 0.00% |          | Rows 0/382104 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/424087/jobs/named/time_series_hourly_feature_group_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x2142286e6a0>, None)